# This is to test the code of the AL loop

Eventualy will be in a script but there is a lot to put together and I want to be able to run this line by line

In [2]:
from glob import glob
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlflow.models.signature import infer_signature
from datetime import datetime
import os
from finkvra.utils.features import make_features as fvra_make_features
from finkvra.utils.labels import cli_label_one_object as fvra_cli_label_one_object
import json
from mlflow.tracking import MlflowClient
import mlflow
import logging

In [4]:
#EXPERIMENT = "gal_model_AL2"
EXPERIMENT = "SEP25gal_model_AL"
SAMPLING_STRATEGY = "uncertainty"
mlflow.set_tracking_uri("http://127.0.0.1:6969")
mlflow.set_experiment(EXPERIMENT)

<Experiment: artifact_location='mlflow-artifacts:/575240719855823327', creation_time=1759241745077, experiment_id='575240719855823327', last_update_time=1759241745077, lifecycle_stage='active', name='SEP25gal_model_AL', tags={}>

If get ``[WARNING] .... Errno 111 Connection refused`` need to make sure we start the server

```bash
mlflow server --host 127.0.0.1 --port 6969
```

In [5]:
# --------------
# 0. Set up 
# --------------

client = MlflowClient()
experiment = client.get_experiment_by_name(EXPERIMENT)

In [6]:

experiment_id = experiment.experiment_id

runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=["start_time DESC"],
    max_results=1,
)

if not runs:
    raise RuntimeError("No successful (FINISHED) runs found.")

last_run = runs[0]
prev_run_id = last_run.info.run_id
print("Last successful run ID:", prev_run_id)


Last successful run ID: 16d68b3e880743b582a32d1a28ef4be5


In [7]:
# -------------------
# 1. Load data
# -------------------
files = sorted(glob('/home/stevance/Data/FinkZTFStream/*.parquet'))
dfs = [pd.read_parquet(f) for f in files]
data = pd.concat(dfs, ignore_index=True)


In [8]:
data.head(2)

,candid,objectId,ra,dec,drb,mjd,mag,maglim,fid,isdiffpos,lc_features_g,lc_features_r,sherl_class,sep_arcsec
0,3069386415315015051,ZTF25aastlyj,288.662154,65.088530,0.961113,"[60823.38641199982, 60795.374525499996, 60795....","[19.527942657470703, nan, nan, nan, nan, nan, ...","[20.746294021606445, 20.559799194335938, 20.42...","[1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, ...","[t, None, None, None, None, None, None, None, ...","{'amplitude': 0.0, 'anderson_darling_normal': ...","{'amplitude': None, 'anderson_darling_normal':...",ORPHAN,nan
1,3069387894915010012,ZTF25aastmkd,341.846733,72.183242,0.994998,"[60823.38789349981, 60795.426064800005, 60795....","[19.923898696899414, nan, nan, nan, nan, nan, ...","[20.43282127380371, 20.07029914855957, 20.3325...","[1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, ...","[f, None, None, None, None, None, None, None, ...","{'amplitude': 0.0, 'anderson_darling_normal': ...","{'amplitude': None, 'anderson_darling_normal':...",ORPHAN,nan


In [9]:
# -------------------
# 2. Make our features
# -------------------
X, meta = fvra_make_features(data)
# remove samples that have no positive detections
valid_candid = list(X[X.ndets > 0].index.values)
X = X.loc[valid_candid]
meta = meta.loc[valid_candid]


In [10]:
X.shape[0], meta.shape[0]

(6217, 6217)

In [18]:
X.head()

,ra,dec,drb,ndets,nnondets,dets_median,dets_std,sep_arcsec,amplitude,linear_fit_reduced_chi2,...,linear_fit_slope_sigma,median,median_absolute_deviation,amplituder_,linear_fit_reduced_chi2r_,linear_fit_sloper_,linear_fit_slope_sigmar_,medianr_,median_absolute_deviationr_,ebv
candid,,,,,,,,,,,,,,,,,,,,,
3069386415315015051,288.662154,65.088530,0.961113,1.0,17.0,19.527943,0.000000,nan,0.000000,NaN,...,NaN,19.527943,0.000000,0.000000,NaN,NaN,NaN,19.527943,0.000000,0.108660
3069382582215015014,293.757494,24.732624,0.999734,1.0,17.0,19.765827,0.000000,1.2398935354229732,0.000000,NaN,...,NaN,19.765827,0.000000,0.000000,NaN,NaN,NaN,19.765827,0.000000,1.916294
3069386410515015007,286.972916,60.047391,0.999990,11.0,6.0,20.475401,0.075382,0.42265986534096434,0.070496,0.069703,...,0.004152,19.842197,0.018932,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932,0.075731
3069388402015015013,334.137737,46.956847,0.995750,1.0,16.0,19.520535,0.000000,9.0563054064397,0.000000,NaN,...,NaN,19.520535,0.000000,0.000000,NaN,NaN,NaN,19.520535,0.000000,0.159215
3069388403815015014,332.785467,48.254230,0.999997,5.0,11.0,19.384800,0.150037,nan,0.152000,0.426425,...,0.047245,19.384800,0.036600,0.152000,0.426425,0.069098,0.047245,19.384800,0.036600,0.282372


In [19]:
meta.head()

,objectId
candid,
3069386415315015051,ZTF25aastlyj
3069382582215015014,ZTF25aastlfp
3069386410515015007,ZTF19aarxyde
3069388402015015013,ZTF25aastlnm
3069388403815015014,ZTF21abiumls


In [13]:
# -------------------
# 3. Load previous training set and exclude
# -------------------
# Artifact: we log and load training IDs as a CSV
previous_ids_path = "./training_ids.csv"
previous_ids_df = pd.read_csv(previous_ids_path)
CURRENT_ROUND= previous_ids_df.iloc[-1]['round'] + 1

train_ids = previous_ids_df["candid"].tolist()
X_pool = X.drop(index=train_ids, errors='ignore')
meta_pool = meta.drop(index=train_ids, errors='ignore')

In [14]:
previous_ids_df

,candid,round
0,3069386410515015007,0
1,3069382582215015014,0
2,3069388402015015013,0
3,3069382583515015017,0
4,3069386414215015010,0
5,3069387892715015046,0
6,3069383052515015018,0
7,3069388403815015014,0
8,3070411123815015010,0
9,3070413103915015007,0


In [15]:
X_pool.shape

(6157, 21)

In [16]:
mlflow_uri = mlflow.get_tracking_uri()

In [17]:
prev_run_id

'16d68b3e880743b582a32d1a28ef4be5'

In [20]:
# -------------------
# 4. Load previous model and predict
# -------------------
model_subpath = "gal_model"
model_uri = f"runs:/{prev_run_id}/{model_subpath}"
clf = mlflow.sklearn.load_model(model_uri)
y_pred = clf.predict_proba(X_pool)[:, 1]  # assuming binary classifications


This will hang forever if the files don't exist

In [21]:
y_pred_pool = pd.DataFrame( np.vstack((X_pool.index.values.astype(str), y_pred)).T, columns= ['candid', 'pred']).set_index('candid')

In [22]:
y_pred_pool.pred

candid
3069386415315015051    0.5666666666666667
3069386413215015028    0.5666666666666667
3069383051615015019    0.5666666666666667
3069383054815015012    0.5666666666666667
3069386413215015015    0.5666666666666667
                              ...        
3190297752615015055    0.5666666666666667
3190298226315015028    0.5666666666666667
3190298221915015018    0.5666666666666667
3190302020115015008    0.5666666666666667
3190302025915015010    0.5666666666666667
Name: pred, Length: 6157, dtype: object

In [23]:
# -------------------
# 6. Active Learning sampling
# -------------------
y_pred_pool['uncertainty_score'] = np.abs(y_pred_pool["pred"].astype(float) - 0.5) 

In [24]:
y_pred_pool = y_pred_pool.sort_values('uncertainty_score', ascending=True)
y_pred_pool

,pred,uncertainty_score
candid,,
3069386415315015051,0.5666666666666667,0.066667
3134442471915015023,0.5666666666666667,0.066667
3134445771015015024,0.5666666666666667,0.066667
3134442471915015016,0.5666666666666667,0.066667
3134444360215015002,0.5666666666666667,0.066667
...,...,...
3122296204115015012,0.5666666666666667,0.066667
3122299544415015015,0.5666666666666667,0.066667
3122294792615015006,0.5666666666666667,0.066667


In [25]:
# -------------------
# 7. Labels 
# -------------------

labels = pd.read_csv('~/Data/FinkZTFStream/labeled.csv', index_col=0)

In [26]:
labels

,objectId,label,timestamp
candid,,,
3069386410515015007,ZTF19aarxyde,varstar,2025-06-02T08:37:22.993787
3069386415315015051,ZTF25aastlyj,real,2025-06-02T08:38:09.167712
3069382582215015014,ZTF25aastlfp,varstar,2025-06-02T08:38:35.964541
3069388402015015013,ZTF25aastlnm,varstar,2025-06-02T08:38:48.162550
3069382583515015017,ZTF25aastmlg,varstar,2025-06-02T09:44:44.227317
...,...,...,...
3072413572515015193,ZTF25aatifwl,real,2025-06-05T13:14:19.761350
3072419752515015002,ZTF25aasdbss,extragal,2025-06-05T13:14:19.761350
3072412154615015021,ZTF18acvwtyw,gal,2025-06-05T13:14:19.761350


In [27]:
label2galclass = {'real': np.nan, 
                  'extragal': 0, 
                  'gal': 1, 
                  'agn': 0,
                  'bogus': np.nan, 
                  'varstar': 1,
                  None: np.nan
                 }

In [28]:
new_labels = []
new_label_candid = []
new_sample_candid = []

N_to_sample = 10
N_i = 0

for _candid in y_pred_pool.index.astype(np.int64):
    try: 
        classification = label2galclass[labels.loc[_candid].label]

        if not np.isnan(classification):
            new_sample_candid.append(_candid)
            N_i += 1
    except KeyError: 
        # if _candid not in labels then labels.loc[_candid] will throw a KeyError 
        # and we can activate the logic below
        _objectId = meta.loc[_candid].objectId
        
        # this is where we need the labeling 
        label = fvra_cli_label_one_object(_objectId)
        if label is None: 
            continue
            
        new_labels.append(label)
        new_label_candid.append(_candid)
        classification = label2galclass[label]
        if not np.isnan(classification):
            new_sample_candid.append(_candid)
            N_i += 1
        
    if N_i == N_to_sample:
        break

if N_i < N_to_sample:
    print('not enough ')


 Object: ZTF25abgofql
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abgofql...
Label [r/x/g/a/b/v] (s=skip, q=quit): r
Labeled as 'real'

 Object: ZTF25abcanrv
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abcanrv...
Label [r/x/g/a/b/v] (s=skip, q=quit): x
Labeled as 'extragal'

 Object: ZTF25abgofqd
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abgofqd...
Label [r/x/g/a/b/v] (s=skip, q=quit): r
Labeled as 'real'

 Object: ZTF25abbzsku
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abbzsku...
Label [r/x/g/a/b/v] (s=skip, q=quit): x
Labeled as 'extragal'

 Object: ZTF23aaorfvb
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF23aaorfvb...
Label [r/x/g/a/b/v] (s=skip, q=quit): v
Labeled as 'varstar'

 Object: ZTF25abgahjn
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abgahjn...
Label [r/x/g/a/b/v] (s=skip, q=quit): x
Labeled as 'extragal'

 Object: ZTF25abaariq
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25abaariq...
Label [r/x/g/a/b/v] (s=skip, q=quit): r
La

In [29]:
N_i

10

In [30]:
y_pred_pool.index.astype(np.int64)

Index([3069386415315015051, 3134442471915015023, 3134445771015015024,
       3134442471915015016, 3134444360215015002, 3134442952415015026,
       3134441444515015010, 3134443410815015012, 3134441445415015007,
       3134441440115015035,
       ...
       3122298122315015012, 3122295742315015011, 3122297172015015027,
       3122297652015015007, 3122294795215010000, 3122296204115015012,
       3122299544415015015, 3122294792615015006, 3118216714315015026,
       3190302025915015010],
      dtype='int64', name='candid', length=6157)

In [31]:
meta

,objectId
candid,
3069386415315015051,ZTF25aastlyj
3069382582215015014,ZTF25aastlfp
3069386410515015007,ZTF19aarxyde
3069388402015015013,ZTF25aastlnm
3069388403815015014,ZTF21abiumls
...,...
3190297752615015055,ZTF25abubaij
3190298226315015028,ZTF25abqvlpi
3190298221915015018,ZTF25ablznzy


In [32]:
timestamp = datetime.utcnow().isoformat()
new_labels_df = pd.DataFrame.from_dict({
                                         'objectId': meta.loc[np.array(new_label_candid).astype(np.int64)].objectId,
                                          'label':  new_labels,
                                        'timestamp': timestamp
                                       })

In [33]:
new_labels_df

,objectId,label,timestamp
candid,,,
3134442471915015023,ZTF25abgofql,real,2025-09-30T14:29:17.551981
3134445771015015024,ZTF25abcanrv,extragal,2025-09-30T14:29:17.551981
3134442471915015016,ZTF25abgofqd,real,2025-09-30T14:29:17.551981
3134444360215015002,ZTF25abbzsku,extragal,2025-09-30T14:29:17.551981
3134442952415015026,ZTF23aaorfvb,varstar,2025-09-30T14:29:17.551981
3134441444515015010,ZTF25abgahjn,extragal,2025-09-30T14:29:17.551981
3134443410815015012,ZTF25abaariq,real,2025-09-30T14:29:17.551981
3134441445415015007,ZTF24aasbqex,varstar,2025-09-30T14:29:17.551981
3134441440115015035,ZTF25abgoeto,real,2025-09-30T14:29:17.551981


In [34]:
updated_labels = pd.concat((labels, new_labels_df))
updated_labels

,objectId,label,timestamp
candid,,,
3069386410515015007,ZTF19aarxyde,varstar,2025-06-02T08:37:22.993787
3069386415315015051,ZTF25aastlyj,real,2025-06-02T08:38:09.167712
3069382582215015014,ZTF25aastlfp,varstar,2025-06-02T08:38:35.964541
3069388402015015013,ZTF25aastlnm,varstar,2025-06-02T08:38:48.162550
3069382583515015017,ZTF25aastmlg,varstar,2025-06-02T09:44:44.227317
...,...,...,...
3134442471915015012,ZTF25abgofqq,bogus,2025-09-30T14:29:17.551981
3134440420615015020,ZTF25abgoecw,bogus,2025-09-30T14:29:17.551981
3134441440115015017,ZTF25abgoetl,real,2025-09-30T14:29:17.551981


In [35]:
updated_labels.to_csv('~/Data/FinkZTFStream/labeled.csv')

In [36]:
new_ids_df = pd.DataFrame({'candid': np.array(new_sample_candid).astype(np.int64),
                           'round': CURRENT_ROUND,
                          })
new_ids_df

,candid,round
0,3134445771015015024,3
1,3134444360215015002,3
2,3134442952415015026,3
3,3134441444515015010,3
4,3134441445415015007,3
5,3134444832415015013,3
6,3134442472415015019,3
7,3134440931315015014,3
8,3134441442415015018,3
9,3134446710615015012,3


In [37]:
previous_ids_df

,candid,round
0,3069386410515015007,0
1,3069382582215015014,0
2,3069388402015015013,0
3,3069382583515015017,0
4,3069386414215015010,0
5,3069387892715015046,0
6,3069383052515015018,0
7,3069388403815015014,0
8,3070411123815015010,0
9,3070413103915015007,0


In [38]:
train_ids_df = pd.concat([previous_ids_df, new_ids_df]).reset_index(drop=True)
train_ids_df.tail(15)

,candid,round
55,3070419442415015034,2
56,3070413571415015007,2
57,3070410150415015023,2
58,3070420384315015019,2
59,3070419441815015028,2
60,3134445771015015024,3
61,3134444360215015002,3
62,3134442952415015026,3
63,3134441444515015010,3
64,3134441445415015007,3


In [39]:
train_ids_df.to_csv('./training_ids.csv', index=False)


train_ids_df.to_csv

In [40]:
y_train = updated_labels.loc[train_ids_df.candid].label.map(label2galclass)

In [41]:
X_train = X.loc[train_ids_df.candid]

In [42]:
X_train

,ra,dec,drb,ndets,nnondets,dets_median,dets_std,sep_arcsec,amplitude,linear_fit_reduced_chi2,...,linear_fit_slope_sigma,median,median_absolute_deviation,amplituder_,linear_fit_reduced_chi2r_,linear_fit_sloper_,linear_fit_slope_sigmar_,medianr_,median_absolute_deviationr_,ebv
candid,,,,,,,,,,,,,,,,,,,,,
3069386410515015007,286.972916,60.047391,0.999990,11.0,6.0,20.475401,0.075382,0.42265986534096434,0.070496,0.069703,...,0.004152,19.842197,0.018932,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932,0.075731
3069382582215015014,293.757494,24.732624,0.999734,1.0,17.0,19.765827,0.000000,1.2398935354229732,0.000000,NaN,...,NaN,19.765827,0.000000,0.000000,NaN,NaN,NaN,19.765827,0.000000,1.916294
3069388402015015013,334.137737,46.956847,0.995750,1.0,16.0,19.520535,0.000000,9.0563054064397,0.000000,NaN,...,NaN,19.520535,0.000000,0.000000,NaN,NaN,NaN,19.520535,0.000000,0.159215
3069382583515015017,296.641470,26.630707,0.951255,1.0,16.0,19.625126,0.000000,3.1566587143809444,0.000000,NaN,...,NaN,19.625126,0.000000,0.000000,NaN,NaN,NaN,19.625126,0.000000,2.957513
3069386414215015010,283.759268,62.913423,0.999961,1.0,16.0,20.100019,0.000000,5.758135128184068,0.000000,NaN,...,NaN,19.690671,0.000000,0.000000,NaN,NaN,NaN,19.690671,0.000000,0.079199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134444832415015013,354.118169,-3.564520,0.999749,54.0,16.0,20.179150,0.180819,0.32485628375372544,0.245444,0.302951,...,0.003115,19.837105,0.067238,0.245444,0.302951,-0.008631,0.003115,19.837105,0.067238,0.032461
3134442472415015019,340.404696,-3.177226,1.000000,88.0,4.0,18.609800,0.519628,1.731309878457182,0.306903,0.902302,...,0.001702,19.387749,0.055922,0.306903,0.902302,0.017201,0.001702,19.387749,0.055922,0.052012
3134440931315015014,335.928744,16.233610,0.999993,26.0,7.0,20.010553,0.113595,0.12890749399105816,0.082472,0.277111,...,0.003319,19.621322,0.038970,0.082472,0.277111,-0.000540,0.003319,19.621322,0.038970,0.055347


In [43]:
X_train.shape, y_train.shape

((70, 21), (70,))

In [48]:
X.columns

Index(['ra', 'dec', 'drb', 'ndets', 'nnondets', 'dets_median', 'dets_std',
       'sep_arcsec', 'amplitude', 'linear_fit_reduced_chi2',
       'linear_fit_slope', 'linear_fit_slope_sigma', 'median',
       'median_absolute_deviation', 'amplituder_', 'linear_fit_reduced_chi2r_',
       'linear_fit_sloper_', 'linear_fit_slope_sigmar_', 'medianr_',
       'median_absolute_deviationr_', 'ebv'],
      dtype='object')

### Train and log with ML Flow

In [44]:
CURRENT_ROUND

3

In [45]:
MODEL_TAG = f"gal_model_round_{CURRENT_ROUND}"

with mlflow.start_run(run_name=f"round_{CURRENT_ROUND}_{SAMPLING_STRATEGY}"):

    # Log metadata
    meta_info = {
        "round": int(CURRENT_ROUND),
        "timestamp": datetime.utcnow().isoformat(),
        "n_train": int(X_train.shape[0]),
        "sampling_strategy": str(SAMPLING_STRATEGY),
        "model_tag": str(MODEL_TAG)
    }
    with open("meta.json", "w") as f:
        json.dump(meta_info, f, indent=2)
    mlflow.log_artifact("meta.json")

    # Train model
    clf_new = HistGradientBoostingClassifier(max_iter=100, 
                                             l2_regularization=10,
                                             random_state=42,
                                             learning_rate=0.1)
    clf_new.fit(X_train.values, y_train.values)

    # Evaluate on training set
    acc = accuracy_score(y_train, clf_new.predict(X_train.values))
    mlflow.log_metric("accuracy", acc)
    
    prec = precision_score(y_train, clf_new.predict(X_train.values))
    mlflow.log_metric("precision", prec)
    
    recall = recall_score(y_train, clf_new.predict(X_train.values))
    mlflow.log_metric("recall", recall)
    
    f1 = f1_score(y_train, clf_new.predict(X_train.values))
    mlflow.log_metric("f1-score", f1)

    # Log model
    signature = infer_signature(X_train, clf_new.predict(X_train))
    mlflow.sklearn.log_model(
        clf_new,
        artifact_path="gal_model",
        signature=signature,
        input_example=X_train.iloc[:2]
    )

    # Save training state
    mlflow.log_artifact("training_ids.csv")

/home/stevance/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


🏃 View run round_3_uncertainty at: http://127.0.0.1:6969/#/experiments/575240719855823327/runs/4e2766df1a5f466f9aab9d5eb9d62c64
🧪 View experiment at: http://127.0.0.1:6969/#/experiments/575240719855823327


/home/stevance/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(
